# Realtime APIを使ったデータ集約型アプリケーションの実践ガイド

このクックブックは、AIエンジニアがOpenAIのRealtime APIの効果を最大化するための実践ガイドとして機能します。特に、データ集約型の関数呼び出しを扱う際に焦点を当てています。音声対音声エージェントでよく見られるシナリオにおいて、大量のデータをスムーズかつ効率的に処理する必要がある場面を重視します。

この記事では、Realtime APIソリューションの基本的なセットアップについては説明しません。代わりに、リアルタイム会話エージェントのパフォーマンスと信頼性を向上させるための明確な洞察と実行可能な戦略を得ることができます。リアルタイム会話コンテキストにおいて大量のデータを処理する際に特有の課題に対処します。

### Realtime APIとは？

詳しく説明する前に、初めての方のためにAPIについて簡単におさらいしましょう。OpenAI Realtime APIは最近提供された機能で、低遅延でマルチモーダルなインタラクション（音声から音声への会話やライブ転写など）をサポートします。リアルタイムの音声ベースカスタマーサポートやライブ映画転写などのシナリオを想像してみてください。

### データ集約的な関数呼び出しとは何か？

エージェントはタスクを実行するために、ツールと関連データへのアクセスが必要です。例えば、金融アナリストエージェントはリアルタイムの市場データを取得する場合があります。多くの場合、あなたの環境には既にAPIを通じてこの情報を公開するサービスが存在しています。

歴史的に、APIはエージェントを念頭に置いて設計されておらず、サービスによっては大量のデータを返すことがよくあります。エンジニアとして、私たちはエージェント開発を加速するために、これらのAPIを関数呼び出しでラップすることがよくあります。これは完全に理にかなっています。既に存在するものをなぜ再発明する必要があるでしょうか？

慎重に最適化されていない場合、これらのデータ集約的な関数呼び出しは、Realtime APIを素早く圧迫し、応答の遅延やユーザーリクエストの処理失敗を引き起こす可能性があります。

### 準備段階

この例では、複数の関数を呼び出して今後のドラフト候補の詳細な分析を提供するNBAスカウティングエージェントを中心に説明します。Realtime APIのやり取りに関する実用的なガイドラインを実証するために、NBAドラフト候補からインスピレーションを得た大規模で現実的なペイロードを使用します。以下では、準備段階として、Realtimeセッションで定義されたモノリシックな`searchDraftProspects`関数を紹介します。

```json
// "Hey, pull up point guards projected in the top 10 in the 2025 draft"
// 「2025年ドラフトでトップ10に予想されているポイントガードを表示して」
{
  "type": "session.update",
  "session": {
    "tools": [
      {
        "type": "function",
        "name": "searchDraftProspects",
        "description": "指定された年のドラフト候補を検索する（例：ポイントガード）",
        "parameters": {
          "type": "object",
          "properties": {
            "sign": {
              "type": "string",
              "description": "選手のポジション",
              "enum": [
                "Point Guard",
                "Shooting Guard",
                "Small Forward",
                "Power Forward",
                "Center",
                "Any"
              ]
            },
            year: { type: "number", description: "ドラフト年（例：2025）" },
            mockDraftRanking: { type: "number", description: "予想ドラフト順位" },
          },
          "required": ["position", "year"]
        }
      }
    ],
    "tool_choice": "auto",
  }
}
```

`searchDraftProspects`関数の呼び出しは大量のペイロードを返します。この例の構造とサイズは、私たちが実際に遭遇した現実世界のシナリオから引用されています。

```json
// ペイロードの例
{
  "status": {
    "code": 200,
    "message": "SUCCESS"
  },
  "found": 4274,
  "offset": 0,
  "limit": 10,
  "data": [
    {
      "prospectId": 10001,
      "data": {
        "ProspectInfo": {
          "league": "NCAA",
          "collegeId": 301,
          "isDraftEligible": true,
          "Player": {
            "personalDetails": {
              "firstName": "Jalen",
              "lastName": "Storm",
              "dateOfBirth": "2003-01-15",
              "nationality": "USA"
            },
            "physicalAttributes": {
              "position": "PG",
              "height": {
                "feet": 6,
                "inches": 4
              },
              "weightPounds": 205
            },
            "hometown": {
              "city": "Springfield",
              "state": "IL"
            }
          },
          "TeamInfo": {
            "collegeTeam": "Springfield Tigers",
            "conference": "Big West",
            "teamRanking": 12,
            "coach": {
              "coachId": 987,
              "coachName": "Marcus Reed",
              "experienceYears": 10
            }
          }
        },
        "Stats": {
          "season": "2025",
          "gamesPlayed": 32,
          "minutesPerGame": 34.5,
          "shooting": {
            "FieldGoalPercentage": 47.2,
            "ThreePointPercentage": 39.1,
            "FreeThrowPercentage": 85.6
          },
          "averages": {
            "points": 21.3,
            "rebounds": 4.1,
            "assists": 6.8,
            "steals": 1.7,
            "blocks": 0.3
          }
        },
        "Scouting": {
          "evaluations": {
            "strengths": ["Court vision", "Clutch shooting"],
            "areasForImprovement": ["Defensive consistency"]
          },
          "scouts": [
            {
              "scoutId": 501,
              "name": "Greg Hamilton",
              "organization": "National Scouting Bureau"
            }
          ]
        },
        "DraftProjection": {
          "mockDraftRanking": 5,
          "lotteryPickProbability": 88,
          "historicalComparisons": [
            {
              "player": "Chris Paul",
              "similarityPercentage": 85
            }
          ]
        },
        "Media": {
          "highlightReelUrl": "https://example.com/highlights/jalen-storm",
          "socialMedia": {
            "twitter": "@jstorm23",
            "instagram": "@jstorm23_ig"
          }
        },
        "Agent": {
          "agentName": "Rick Allen",
          "agency": "Elite Sports Management",
          "contact": {
            "email": "rallen@elitesports.com",
            "phone": "555-123-4567"
          }
        }
      }
    },
    // ... 数千トークン後...
  ]
}
```

## 指導原則

### 1. 扱いにくい関数をより小さく、明確な役割と責任を持つ関数に分割する

関数呼び出しを構築する際、最優先事項は明確で適切に定義された関数を設計することです。これは言うまでもないことです。これにより、レスポンスサイズを削減し、モデルを圧迫することを避けやすくなります。各関数呼び出しは説明しやすく、範囲が明確に定められ、その目的に必要な情報のみを返すべきです。関数間で責任が重複すると、必然的に混乱を招きます。

例えば、`searchDraftProspects`関数呼び出しを、各プロスペクトの一般的な詳細（選手統計など）のみを返すように制限することで、レスポンスサイズを劇的に削減できます。より多くの情報が必要な場合は、新しい`getProspectDetails`関数呼び出しが拡張された詳細を提供します。万能な解決策はありません。適切なアプローチは、あなたのユースケースとデータモデルによって決まります。

```json
{
  "tools": [
    {
      "type": "function",
      "name": "searchDraftProspects",
      "description": "Search NBA draft prospects by position, draft year, and projected ranking, returning only general statistics to optimize response size.",
      "parameters": {
        "type": "object",
        "properties": {
          "position": {
            "type": "string",
            "description": "The player's basketball position.",
            "enum": [
              "Point Guard",
              "Shooting Guard",
              "Small Forward",
              "Power Forward",
              "Center",
              "Any"
            ]
          },
          "year": {
            "type": "number",
            "description": "Draft year, e.g., 2025"
          },
          "maxMockDraftRanking": {
            "type": "number",
            "description": "Maximum predicted draft ranking (e.g., top 10)"
          }
        },
        "required": ["position", "year"]
      }
    },
    {
      "type": "function",
      "name": "getProspectDetails",
      "description": "Fetch detailed information for a specific NBA prospect, including comprehensive stats, agent details, and scouting reports.",
      "parameters": {
        "type": "object",
        "properties": {
          "playerName": {
            "type": "string",
            "description": "Full name of the prospect (e.g., Jalen Storm)"
          },
          "year": {
            "type": "number",
            "description": "Draft year, e.g., 2025"
          },
          "includeAgentInfo": {
            "type": "boolean",
            "description": "Include agent information"
          },
          "includeStats": {
            "type": "boolean",
            "description": "Include detailed player statistics"
          },
          "includeScoutingReport": {
            "type": "boolean",
            "description": "Include scouting report details"
          }
        },
        "required": ["playerName", "year"]
      }
    }
  ],
  "tool_choice": "auto"
}
```

### 2. 会話が進行するにつれて、コンテキストを最適化する

リアルタイム会話では30分間の寛大なセッションが可能ですが、ローリングコンテキストウィンドウは約16,000トークンしかサポートしていません（モデルスナップショットによって異なり、コンテキストウィンドウの制限は改善されています）。その結果、長時間の会話交換中にパフォーマンスが徐々に低下することがあります。会話が進行し、より多くの関数呼び出しが行われると、会話状態は重要な情報と不要なノイズの両方で急速に拡大する可能性があります。そのため、最も関連性の高い詳細情報を保持することに焦点を当てることが重要です。このアプローチは、強力なパフォーマンスを維持し、コストを削減するのに役立ちます。

**i) 会話の状態を定期的に要約する**

会話が進行する中で定期的に要約することは、コンテキストサイズを削減する優れた方法です。これによりコストとレイテンシの両方を削減できます。

Realtimeでの会話における自動要約の実装について、@Minhajulの素晴らしいガイドをご覧ください（[リンク](https://cookbook.openai.com/examples/context_summarization_with_realtime_api)）。

**ii) モデルの役割と責任を定期的に思い出させる**

データ量の多いペイロードは、コンテキストウィンドウをすぐに埋めてしまう可能性があります。モデルが指示や利用可能なツールを見失っていることに気づいた場合は、`session.update`を呼び出してシステムプロンプトとツールを定期的に思い出させてください。これにより、モデルがその役割と責任に集中し続けることができます。

### 3. データ処理と最適化

**i) 関数呼び出しでフィルタリングを使用して、データ量の多いレスポンスを質問に答えるために必要な重要なフィールドのみに絞り込む**

一般的に、関数呼び出しから返されるトークン数が少ないほど、より高品質なレスポンスが得られます。よくある落とし穴は、関数呼び出しが数千トークンにわたる過度に大きなペイロードを返すことです。レスポンスサイズを最小化するために、各関数呼び出しでデータレベルまたは関数レベルでフィルターを適用することに焦点を当てましょう。

```json
// フィルタリングされたレスポンス
{
  "status": {
    "code": 200,
    "message": "SUCCESS"
  },
  "found": 4274,
  "offset": 0,
  "limit": 5,
  "data": [
    {
    "zpid": 7972122,
      "data": {
        "PropertyInfo": {
            "houseNumber": "19661",
            "directionPrefix": "N ",
            "streetName": "Central",
            "streetSuffix": "Ave",
            "city": "Phoenix",
            "state": "AZ",
            "postalCode": "85024",
            "zipPlusFour": "1641"
            "bedroomCount": 2,
            "bathroomCount": 2,
            "storyCount": 1,
            "livingAreaSize": 1089,
            "livingAreaSizeUnits": "Square Feet",
            "yearBuilt": "1985"
          }
		    }
			}
		]
		// ... 
}
```

**ii) 階層化されたペイロードを平坦化する—重要な情報を失うことなく**

API呼び出しからの階層化されたペイロードには、「ProspectInfo」や「Stats」のような繰り返されるレベルタイトルが含まれることがあり、これらは余分なノイズを追加し、モデルが処理するのを困難にする可能性があります。データをより効率的にする方法を探る際に、不要なラベルの一部を削除してこれらの構造を平坦化することを試してみるかもしれません。これはパフォーマンスの向上に役立ちますが、特定のユースケースにとって重要な情報を保持することを考慮してください。

```json
// 平坦化されたペイロード
{
  "status": {
    "code": 200,
    "message": "SUCCESS"
  },
  "found": 4274,
  "offset": 0,
  "limit": 2,
  "data": [
    {
      "prospectId": 10001,
      "league": "NCAA",
      "collegeId": 301,
      "isDraftEligible": true,
      "firstName": "Jalen",
      "lastName": "Storm",
      "position": "PG",
      "heightFeet": 6,
      "heightInches": 4,
      "weightPounds": 205,
      "hometown": "Springfield",
      "state": "IL",
      "collegeTeam": "Springfield Tigers",
      "conference": "Big West",
      "teamRanking": 12,
      "coachId": 987,
      "coachName": "Marcus Reed",
      "gamesPlayed": 32,
      "minutesPerGame": 34.5,
      "FieldGoalPercentage": 47.2,
      "ThreePointPercentage": 39.1,
      "FreeThrowPercentage": 85.6,
      "averagePoints": 21.3,
      "averageRebounds": 4.1,
      "averageAssists": 6.8,
      "stealsPerGame": 1.7,
      "blocksPerGame": 0.3,
      "strengths": ["Court vision", "Clutch shooting"],
      "areasForImprovement": ["Defensive consistency"],
      "mockDraftRanking": 5,
      "lotteryPickProbability": 88,
      "highlightReelUrl": "https://example.com/highlights/jalen-storm",
      "agentName": "Rick Allen",
      "agency": "Elite Sports Management",
      "contactEmail": "rallen@elitesports.com"
    },
		...
 }
 ```

**iii) 異なるデータ形式を試す**

データの構造化方法は、モデルがAPIレスポンスを処理し要約する能力に直接的な影響を与えます。私たちの経験では、JSONやYAMLのような明確でキーベースの形式は、Markdownなどの表形式よりもモデルがデータをより正確に解釈するのに役立ちます。特に大きな表は、モデルを圧倒する傾向があり、その結果、流暢性と精度の低い出力になってしまいます。それでも、あなたのユースケースに最適な形式を見つけるために、異なる形式を試してみる価値があります。

```yaml
status:
  code: 200
  message: "SUCCESS"
found: 4274
offset: 0
limit: 10
data:
  - prospectId: 10001
    data:
      ProspectInfo:
        league: "NCAA"
        collegeId: 301
        isDraftEligible: true
        Player:
          firstName: "Jalen"
          lastName: "Storm"
          position: "PG"
          heightFeet: 6
          heightInches: 4
          weightPounds: 205
          hometown: "Springfield"
          state: "IL"
        TeamInfo:
          collegeTeam: "Springfield Tigers"
          conference: "Big West"
          teamRanking: 12
          coachId: 987
          coachName: "Marcus Reed"
      Stats:
        gamesPlayed: 32
        minutesPerGame: 34.5
        FieldGoalPercentage: 47.2
        ThreePointPercentage: 39.1
        FreeThrowPercentage: 85.6
        averagePoints: 21.3
        averageRebounds: 4.1
        averageAssists: 6.8
        stealsPerGame: 1.7
        blocksPerGame: 0.3
      Scouting:
        strengths:
          - "Court vision"
          - "Clutch shooting"
        areasForImprovement:
          - "Defensive consistency"
      DraftProjection:
        mockDraftRanking: 5
        lotteryPickProbability: 88
      Media:
        highlightReelUrl: "https://example.com/highlights/jalen-storm"
      Agent:
        agentName: "Rick Allen"
        agency: "Elite Sports Management"
        contactEmail: "rallen@elitesports.com"
```

## 4. データ集約的な関数呼び出しの後は、ヒントプロンプトでフォローアップする

基盤となるモデルは、データ集約的な応答から正確な回答への滑らかな移行に苦労することがよくあります。複雑なデータを扱う際の流暢性と正確性を向上させるために、関数呼び出しの直後にファンクションコールヒントを提供してください。これらのヒントは、特定のタスクについてモデルをガイドし、キーフィールドやドメイン固有の値の解釈方法を教えます。

以下の例は、効果的なヒントプロンプトを示しています。

```javascript
// Function call hint
let prospectSearchPrompt = `
Parse NBA prospect data and provide a concise, engaging response.

General Guidelines
- Act as an NBA scouting expert.
- Highlight key strengths and notable attributes.
- Use conversational language.
- Mention identical attributes once.
- Ignore IDs and URLs.

Player Details
- State height conversationally ("six-foot-eight").
- Round weights to nearest 5 lbs.

Stats & Draft Info
- Round stats to nearest whole number.
- Use general terms for draft ranking ("top-five pick").
Experience
- Refer to players as freshman, sophomore, etc., or mention professional experience.
- Location & TeamMention hometown city and state/country.
- Describe teams conversationally.

Skip (unless asked explicitly)
- Exact birth dates
- IDs
- Agent/contact details
- URLs

Examples
- "Jalen Storm, a dynamic six-foot-four point guard from Springfield, Illinois, averages 21 points per game."
- "Known for his clutch shooting, he's projected as a top-five pick."

Important: Respond based strictly on provided data, without inventing details.
`;
```

実際には、まず関数呼び出しの結果を会話に追加します。その後、ヒントプロンプトと共にRealtime APIからレスポンスを出力します。これで、モデルがすべての情報を適切に処理できるようになります。

```javascript
// モデル用の新しい会話アイテムを追加
const conversationItem = {
  type: 'conversation.item.create',
  previous_item_id: output.id,
  item: {
    call_id: output.call_id,
    type: 'function_call_output',
    output: `Draft Prospect Search Results: ${result}`
  }
};

dataChannel.send(JSON.stringify(conversationItem));

// ヒントプロンプトを含むモデルからのレスポンスを送信
const event = {
  type: 'response.create',
  conversation: "none",
  response: {
    instructions: prospectSearchPrompt # function call hint
  }
};

dataChannel.send(JSON.stringify(event));
```

## まとめ

Realtime APIを使用して効果的なエージェントを構築することは、継続的な探求と適応のプロセスです。

**主要な推奨事項の要約**

- **データをフィルタリングする：** ユーザーのリクエストやモデルの次のステップに直接関連するフィールドと詳細のみを含める。それ以外は削除する。
- **構造を平坦化し簡素化する：** 深くネストされた冗長なデータを削減する。モデルと人間の両方がスキャンしやすい方法で情報を提示する。
- **明確で構造化された形式を優先する：** 一貫したフィールド名と最小限のノイズを持つJSON（またはYAML）を使用する。データ量の多い応答では大きなテーブルやMarkdownを避ける。
- **ヒントプロンプトでモデルを誘導する：** 大量のデータを返した後、モデルが何を抽出または要約すべきかを正確に説明するターゲットプロンプトでフォローアップする。

覚えておいてください—実験は不可欠です。Realtimeモデルは継続的に改善されており、私たちはRealtime APIを最大限に活用するためのヒントを共有し続けます。